In [1]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

import pandas as pd

import uuid
import chromadb

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key= "enter_your_groq_api_key",
    model_name= "llama-3.3-70b-versatile"
)

response = llm.invoke("The first person land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
loader = WebBaseLoader("https://www.accenture.com/in-en/careers/jobdetails?id=ATCI-5258550-S1927674_en&title=Custom+Software+Engineer")

page_data = loader.load().pop().page_content
print(page_data)






Custom Software Engineer






























































































































Skip to main content
Skip to footer













Menu




 
Accenture


 


Accenture








Close Menu





What we do

 





Back



What we do





Capabilities
Capabilities



Cloud 
					


Customer Service 
					


Cybersecurity 
					


Data and Artificial Intelligence 
					


Digital Engineering and Manufacturing 
					


Ecosystem Partners 
					


Emerging Technology 
					


Finance and Risk Management 
					


Infrastructure and Capital Projects 
					


Learning 
					


Managed Services 
					


Marketing and Experience 
					


Metaverse 
					


Sales and Commerce 
					


Strategy 
					


Supply Chain 
					


Sustainability 
					


Talent and Organization 
					


Technology Transformation 
					





Industries
Industries



Aerospace and Defense 
					


Automotive 
					


Banking 
					


Capital Markets 


In [4]:
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
{
  "role": "Custom Software Engineer",
  "experience": "7.5 years",
  "skills": ["SAP ABAP Development for HANA"],
  "description": "Implement the configuration management plan as directed by the Configuration Lead. Assist in the design of software configuration and customization to meet the business process design and application requirements."
}
```


In [5]:
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Custom Software Engineer',
 'experience': '7.5 years',
 'skills': ['SAP ABAP Development for HANA'],
 'description': 'Implement the configuration management plan as directed by the Configuration Lead. Assist in the design of software configuration and customization to meet the business process design and application requirements.'}

In [6]:
type(json_res)

dict

In [7]:
df = pd.read_csv('project_portfolio_demo.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular, .NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [8]:
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                      metadatas={'links': row['Links']},
                      ids=[str(uuid.uuid4())])

In [9]:
links = collection.query(query_texts=['Experience in python', 'Experience in React'],
                        n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [11]:
job = json_res
job['skills']

['SAP ABAP Development for HANA']

In [12]:
job

{'role': 'Custom Software Engineer',
 'experience': '7.5 years',
 'skills': ['SAP ABAP Development for HANA'],
 'description': 'Implement the configuration management plan as directed by the Configuration Lead. Assist in the design of software configuration and customization to meet the business process design and application requirements.'}

In [13]:
type(job)

dict

In [14]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Vijay Rakkaiah, a business development executive at VMT. VMT is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of VMT 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase VMT's portfolio: {link_list}
        Remember you are Vijay Rakkaiah, BDE at VMT. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Custom Software Engineering Services for Seamless Business Process Integration

Dear Hiring Manager,

I came across your job posting for a Custom Software Engineer with expertise in SAP ABAP Development for HANA, and I'm excited to introduce VMT, a leading AI & Software Consulting company that can fulfill your requirements. With 7.5 years of experience in implementing configuration management plans and designing software configuration and customization, I believe our team can make a significant impact on your business.

At VMT, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts can assist in the design of software configuration and customization to meet your business process design and application requirements. We can implement the configuration management plan as directed by your Configuration Lead, ensuring seamless integra